In [ ]:
!pip install -r ../requirements.txt > ../.log

In [3]:
# from google.cloud import speech_v1p1beta1 as speech
import io
import openai 
from openai import OpenAI
import requests
import time
import whisper
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pytube import YouTube
import os
import whisper
import tempfile
import ollama
import ssl


ModuleNotFoundError: No module named 'whisper'

In [ ]:
def read_api_keys(file_path):
    api_keys = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            api_keys[key] = value.strip('"')
    return api_keys

# Path to the .api-key file
api_key_file_path = '../data/.api_keys'

# Read and parse the .api-key file
api_keys = read_api_keys(api_key_file_path)

# Access the API keys
assemblyai_api_key = api_keys.get('assemblyai')
openai_api_key = api_keys.get('openai_test')


In [ ]:
# Transcribe audio to text
def transcribe_audio_assemblyai(audio_file_path):
    api_key = assemblyai_api_key
    headers = {
        'authorization': api_key,
        'content-type': 'application/json'
    }

    # Upload the audio file
    upload_url = 'https://api.assemblyai.com/v2/upload'
    with open(audio_file_path, 'rb') as audio_file:
        response = requests.post(upload_url, headers=headers, files={'file': audio_file})
    audio_url = response.json()['upload_url']

    # Request transcription
    transcript_url = 'https://api.assemblyai.com/v2/transcript'
    transcript_request = {
        'audio_url': audio_url
    }
    response = requests.post(transcript_url, headers=headers, json=transcript_request)
    transcript_id = response.json()['id']

    # Poll for transcription result
    while True:
        response = requests.get(f'{transcript_url}/{transcript_id}', headers=headers)
        result = response.json()
        if result['status'] == 'completed':
            return result['text']
        elif result['status'] == 'failed':
            raise Exception('Transcription failed')
        time.sleep(5)

def search_song_lyrics(lyrics, access_token):
    base_url = "https://api.genius.com"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    search_url = f"{base_url}/search"
    params = {"q": lyrics}
    response = requests.get(search_url, headers=headers, params=params)
    response_data = response.json()
    
    if response.status_code == 200 and response_data['response']['hits']:
        song_info = response_data['response']['hits'][0]['result']
        song_title = song_info['title']
        artist_name = song_info['primary_artist']['name']
        return song_title, artist_name
    else:
        return None, None

def transcribe_audio_openai(audio_file_path):
    # Initialize the client
    client = OpenAI(api_key=openai_api_key)
    # Upload and transcribe the audio file
    with open(audio_file_path, 'rb') as audio_file:
        response = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )
    # Extract and return the transcription
    return response.text

# Transcribe audio to text using local Whisper model
def transcribe_audio_local(audio_file_path):
    model = whisper.load_model("base")  # or "tiny", "small", "medium", "large"
    result = model.transcribe(audio_file_path)
    return result["text"]

def identify_song_from_lyrics(lyrics):
    # Initialize the client
    client = OpenAI(api_key=openai_api_key)
    # Create the prompt
    prompt = f"Identify the song from the following lyrics:\n\n{lyrics}\n\nSong name:"
    # Make the API call using the new format
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    # Extract and return the response
    return response.choices[0].message.content.strip()

# Identify song from lyrics using Spotify API
def identify_song_from_lyrics_spotify(lyrics, client_id, client_secret):
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))
    results = sp.search(q=lyrics, type='track', limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        song_name = track['name']
        artist_name = track['artists'][0]['name']
        return song_name, artist_name
    else:
        return None, None

def search_song_lyrics_musixmatch(lyrics, api_key):
    api_key = 'your_musixmatch_api_key'
    base_url = "https://api.musixmatch.com/ws/1.1/"
    search_url = f"{base_url}track.search"
    params = {
        "q_lyrics": lyrics,
        "apikey": api_key,
        "s_track_rating": "desc",
        "f_has_lyrics": 1,
        "page_size": 1,
        "page": 1
    }
    response = requests.get(search_url, params=params)
    response_data = response.json()
    
    if response.status_code == 200 and response_data['message']['header']['status_code'] == 200:
        track_list = response_data['message']['body']['track_list']
        if track_list:
            track = track_list[0]['track']
            song_name = track['track_name']
            artist_name = track['artist_name']
            return song_name, artist_name
    return None, None

# Main function
def main(audio_file_path):
    # lyrics = transcribe_audio_assemblyai(audio_file_path)
    lyrics = transcribe_audio_openai(audio_file_path)
    # lyrics = "Shut up. Yes, please. Won't you come on me? I'm right here. Cause I need somebody."
    # lyrics = transcribe_audio_local(audio_file_path)
    print("Extracted Lyrics:", lyrics)
    # song_name = identify_song_from_lyrics(lyrics)
    # print("Identified Song Name:", song_name)

    # # Identify song from lyrics using Spotify API
    # # song_name, artist_name = identify_song_from_lyrics_spotify(lyrics, client_id, client_secret)
    # # song_name, artist_name = search_song_lyrics_musixmatch(lyrics, api_key)
    # if song_name and artist_name:
    #     print(f"Identified Song Name: {song_name} by {artist_name}")
    # else:
    #     print("Could not identify the song.")
    # if song_name and artist_name:
    #     print(f"Identified Song Name: {song_name} by {artist_name}")
    # else:
    #     print("Could not identify the song.")

# Replace with the path to your audio file
audio_file_path = "../data/track_audio.mp3"
client_id = 'your_spotify_client_id'
client_secret = 'your_spotify_client_secret'
main(audio_file_path)

In [4]:
# Disable SSL verification
ssl._create_default_https_context = ssl._create_unverified_context

def recognize_music_from_youtube(youtube_url):
    try:
        # Create a temporary directory to store the audio file
        with tempfile.TemporaryDirectory() as temp_dir:

            # # Download audio from YouTube
            # yt = YouTube(youtube_url)
            # audio_stream = yt.streams.filter(only_audio=True).first()
            
            # # Download to temporary file
            # temp_audio_path = os.path.join(temp_dir, 'temp_audio.mp4')
            # audio_stream.download(output_path=temp_dir, filename='temp_audio.mp4')

            temp_audio_path = "../data/easy.mp3"

            # Transcribe the audio file using Whisper 
            print(whisper.__file__)
            model = whisper.load_model("base")  # or "tiny", "small", "medium", "large"
            result = model.transcribe(temp_audio_path)
            transcription = result["text"]

            print("Transcription:", transcription)
            
            # transcription = "Your sugar, yes, please Won't you come and put it down on me?  I'm right here, 'cause I need Little love, a little sympathy Yeah, you show me good lovin', make it alright Need a little sweetness in my life Your sugar, yes, please Won't you come and put it down on me?"

            # Now use the transcription to identify the song
            prompt = f"Based on these lyrics or audio transcription, what song is this? If it's a song, please provide the song name and artist. If it's not a song, please indicate that it's not music. Here's the transcription:\n\n{transcription}"
            
            # # Initialize the OpenAI client
            # openai.api_key = 'your_openai_api_key'
            # response = openai.ChatCompletion.create(
            #     model="gpt-3.5-turbo",
            #     messages=[
            #         {"role": "system", "content": "You are a music expert. Identify songs based on lyrics or transcribed audio content."},
            #         {"role": "user", "content": prompt}
            #     ]
            # )

             # Use Ollama to recognize the song
            prompt = f"Identify the song based on these lyrics:\n\n{transcription}\n\nProvide the song name and artist."
            response = ollama.chat(model="mistral", messages=[{"role": "user", "content": prompt}])

            return response['message']['content'].strip()
            
            # return response.choices[0].message['content'].strip()
            
    except Exception as e:
        return f"Error processing video: {str(e)}"

# Example usage:
result = recognize_music_from_youtube("https://www.youtube.com/watch?v=YOUR_VIDEO_ID")
print(result)

/opt/anaconda3/envs/music/lib/python3.12/site-packages/whisper.py
Error processing video: module 'whisper' has no attribute 'load_model'


In [ ]:
# !pip freeze | sed 's/ @ file:.*//' > ../requirements.txt